## 基础导入啥的

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.strtree import STRtree

## 1.gvi

The GVI dataset used in this study is derived from the Treepedia project (MIT, 2015), based on Google Street View images captured around 2015. While this may not fully reflect recent greening interventions or urban modifications, GVI is used here as a general proxy for the visual landscape structure. Limitations related to data currency are discussed in Section X.

本研究所用GVI数据来自于MIT Treepedia项目，基于约2015年的Google街景图像。尽管可能无法反映最近绿化或街景变化，但其作为城市空间绿意结构的代表性指标，仍具有参考价值，相关时效性局限将在后文讨论部分说明。

To address gaps in GVI coverage along street segments, a borough-level imputation strategy was applied. Each road segment was first assigned a GVI value based on the average of nearby GVI points within a 20-meter buffer. For segments lacking any such points, the mean GVI value of the corresponding Greater London borough was used as a proxy. All imputed values were flagged to ensure transparency and to support subsequent interpretation.

为解决街道路段内 GVI 数据覆盖不足的问题，本研究采用了基于 Borough（伦敦行政区）划分的分组插值策略。首先，优先使用 20 米缓冲区内的 GVI 点均值赋值给道路段；若缓冲区内未找到任何 GVI 点，则根据该路段所在 Borough 的平均 GVI 值进行插值补全。所有插值段均被标记，以保证分析过程的透明性和后续结果解释的严谨性。

### 先导入一下数据（一直出错啊妈的

In [2]:
boroughs_gdf = gpd.read_file("../data/BoroughShp/borough/borough.shp")
print(boroughs_gdf.columns)

edges_gdf = gpd.read_file("../data/Roads/london_edges_FIXED.gpkg")
print(edges_gdf.columns)

gvi_points = gpd.read_file("../data/Env/greenview_london.json/greenview_london.json")
print(gvi_points.columns)

Index(['NAME', 'GSS_CODE', 'HECTARES', 'NONLD_AREA', 'ONS_INNER', 'SUB_2009',
       'SUB_2006', 'geometry'],
      dtype='object')
Index(['u', 'v', 'key', 'osmid', 'access', 'highway', 'maxspeed', 'name',
       'oneway', 'reversed', 'length', 'lanes', 'ref', 'bridge', 'junction',
       'tunnel', 'width', 'service', 'est_width', 'area', 'geometry'],
      dtype='object')
Index(['FID', 'PntNum', 'panoID', 'panoDate', 'greenView', 'geometry'], dtype='object')


In [3]:
boroughs_gdf = boroughs_gdf[['NAME', 'geometry']].rename(columns={'NAME': 'name'})

In [4]:
# 将 GVI 点赋值给 Borough（我们之后要 fallback）
gvi_points = gvi_points.to_crs(boroughs_gdf.crs)
gvi_with_borough = gpd.sjoin(gvi_points, boroughs_gdf, how='inner', predicate='within')

# 计算 Borough 的平均 GVI 值
borough_gvi_mean = gvi_with_borough.groupby('name')['greenView'].mean().to_dict()
print(borough_gvi_mean)

{'Barking and Dagenham': 9.956715211543225, 'Barnet': 17.183731562227795, 'Bexley': 12.555201401160128, 'Brent': 13.387036045154845, 'Bromley': 17.490392798331015, 'Camden': 15.705790953840856, 'City of London': 6.078575598230088, 'Croydon': 17.42361365029124, 'Ealing': 14.22662851005025, 'Enfield': 14.374663562873259, 'Greenwich': 15.130032851144279, 'Hackney': 11.230470191756273, 'Hammersmith and Fulham': 12.67179935098781, 'Haringey': 14.22248786059309, 'Harrow': 14.789230103879849, 'Havering': 13.943786303221732, 'Hillingdon': 16.309627819518948, 'Hounslow': 14.003835840731503, 'Islington': 13.337145922688226, 'Kensington and Chelsea': 13.010757235919234, 'Kingston upon Thames': 15.40256658454591, 'Lambeth': 13.578280558524852, 'Lewisham': 13.024022477114041, 'Merton': 15.501925656580617, 'Newham': 11.146743481980197, 'Redbridge': 12.134484017713365, 'Richmond upon Thames': 18.68832873456003, 'Southwark': 14.326239796140351, 'Sutton': 17.120068456362556, 'Tower Hamlets': 12.8026552

In [6]:
# === 1. 创建每条路段的中点列 ===
print("1begin")
edges_gdf['midpoint'] = edges_gdf.geometry.interpolate(0.5, normalized=True)
midpoints_gdf = gpd.GeoDataFrame(edges_gdf[['midpoint']], geometry='midpoint', crs=edges_gdf.crs)

# === 2. 建立 GVI 空间索引，提升效率 ===
print("2begin")
gvi_points = gvi_points.to_crs(edges_gdf.crs)
gvi_sindex = gvi_points.sindex

# === 3. 对每条道路构建 20 米缓冲区并查找 GVI 点 ===
print("3begin")
from tqdm.notebook import tqdm
tqdm.pandas()  # 开启 pandas 专用进度条

def compute_gvi_buffer_mean(point):
    buffer = point.buffer(20)
    possible = list(gvi_sindex.intersection(buffer.bounds))
    near = gvi_points.iloc[possible]
    near = near[near.intersects(buffer)]
    if len(near) > 0:
        return near['greenView'].mean()
    else:
        return None

midpoints_gdf['gvi_buffer_mean'] = midpoints_gdf['midpoint'].progress_apply(compute_gvi_buffer_mean)

# === 4. 给中点赋 Borough 标签，用于 fallback ===
print("4begin")
midpoints_with_borough = gpd.sjoin(midpoints_gdf, boroughs_gdf, how='left', predicate='within')
midpoints_with_borough['borough'] = midpoints_with_borough['name']

# === 5. 建立最终 gvi 值和标记列 ===
print("5begin")
def assign_final_gvi(row):
    if pd.notnull(row['gvi_buffer_mean']):
        return row['gvi_buffer_mean'], 0
    else:
        borough = row['borough']
        return borough_gvi_mean.get(borough, None), 1

midpoints_with_borough[['gvi_final', 'gvi_flag']] = midpoints_with_borough.apply(assign_final_gvi, axis=1, result_type='expand')

# === 6. 合并结果回 edges_gdf ===
print("6begin")
edges_gdf['gvi_final'] = midpoints_with_borough['gvi_final']
edges_gdf['gvi_flag'] = midpoints_with_borough['gvi_flag']

1begin
2begin
3begin


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39456\1382637226.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edges_gdf['midpoint'] = edges_gdf.geometry.interpolate(0.5, normalized=True)


  0%|          | 0/292703 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

# === 0. 只抽取前 3000 条边作为测试集 ===
edges_sample = edges_gdf.iloc[:3000].copy()

# === 1. 创建每条路段的中点列 ===
print("1begin")
edges_sample['midpoint'] = edges_sample.geometry.interpolate(0.5, normalized=True)
midpoints_gdf = gpd.GeoDataFrame(edges_sample[['midpoint']], geometry='midpoint', crs=edges_sample.crs)

# === 2. 建立 GVI 空间索引，提升效率 ===
print("2begin")
gvi_points = gvi_points.to_crs(edges_sample.crs)
gvi_sindex = gvi_points.sindex

# === 3. 对每条道路构建 20 米缓冲区并查找 GVI 点 ===
print("3begin")
def compute_gvi_buffer_mean(point):
    buffer = point.buffer(20)
    possible = list(gvi_sindex.intersection(buffer.bounds))
    near = gvi_points.iloc[possible]
    near = near[near.intersects(buffer)]
    if len(near) > 0:
        return near['greenView'].mean()
    else:
        return None

midpoints_gdf['gvi_buffer_mean'] = midpoints_gdf['midpoint'].progress_apply(compute_gvi_buffer_mean)

# === 4. 给中点赋 Borough 标签，用于 fallback ===
print("4begin")
midpoints_with_borough = gpd.sjoin(midpoints_gdf, boroughs_gdf, how='left', predicate='within')
midpoints_with_borough['borough'] = midpoints_with_borough['name']

# === 5. 建立最终 gvi 值和标记列 ===
print("5begin")
def assign_final_gvi(row):
    if pd.notnull(row['gvi_buffer_mean']):
        return row['gvi_buffer_mean'], 0
    else:
        borough = row['borough']
        return borough_gvi_mean.get(borough, None), 1

midpoints_with_borough[['gvi_final', 'gvi_flag']] = midpoints_with_borough.apply(assign_final_gvi, axis=1, result_type='expand')

# === 6. 合并结果回 edges_sample ===
print("6begin")
edges_sample['gvi_final'] = midpoints_with_borough['gvi_final']
edges_sample['gvi_flag'] = midpoints_with_borough['gvi_flag']

# === 7. 可选：保存小样本结果 ===
edges_sample.to_file("output/edges_gvi_sample_result.gpkg", layer='edges', driver="GPKG")


1begin
2begin
3begin


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39456\802947924.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edges_sample['midpoint'] = edges_sample.geometry.interpolate(0.5, normalized=True)


  0%|          | 0/3000 [00:00<?, ?it/s]

KeyboardInterrupt: 